In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from bm25.bm25 import BM25Search

# class BM25Search(BaseSearch): 
#     # keys - поля в collection (кроме индекса!)
#     def __init__(self, index_name: str, hostname: str = "localhost", keys: Dict[str, str] =  {"body": "txt"}, language: str = "russian",
#                  batch_size: int = 128, timeout: int = 1000, retry_on_timeout: bool = True, maxsize: int = 24, number_of_shards: int = "default",
#                  initialize: bool = True, sleep_for: int = 2)
from bm25.evaluation import EvaluateRetrieval

hostname = "http://localhost:9200"
index_name = "mmarco" 
language = "russian"


bm25 = BM25Search(index_name=index_name, hostname=hostname, language=language, keys={"body":"text"})
retriever = EvaluateRetrieval(bm25,k_values=[1]) 

In [3]:
# Create a small test corpus with document IDs and content
corpus = {
    "doc1": {
        "text": "Озеро Байкал отличается уникальной флорой. В озере обитает более 2500 видов растений, из которых 1000 являются эндемиками."
    },
    "doc2": {
        "text": "В водах Байкала обитает множество уникальных животных. Байкальская нерпа и голомянка являются эндемиками озера."
    },
    "doc3": {
        "text": "Байкал - самое глубокое озеро в мире. Его максимальная глубина достигает 1642 метров, а площадь поверхности составляет 31722 км²."
    }
}

# Create test queries with query IDs
queries = {
    "query1": "где находится столица России",
    "query2": "кто основал Санкт-Петербург",
    "query3": "глубокое озеро в мире",
    "query4": "история озера в санкт-петербурге"
}




In [4]:
results = retriever.retrieve(corpus, queries)

print(results) #{'query3': {'doc3': 2.462403, 'doc1': 0.593623, 'doc2': 0.14181954}, 'query4': {'doc2': 1.407097, 'doc1': 0.5128078, 'doc3': 0.12818062}}

que: 100%|██████████| 1/1 [00:00<00:00, 61.69it/s]

{'query3': {'doc3': 2.462403, 'doc1': 0.593623}, 'query4': {'doc2': 1.1835277, 'doc1': 0.13133836}}


In [31]:
# Create qrels (query relevance judgments)
# Format: qrels[query_id][doc_id] = relevance_score
# Relevance scores: 0 = not relevant, 1 = relevant
qrels = {
    "query1": {
        "doc1": 1,    # Relevant - about Moscow being capital
        "doc2": 0,    # Not relevant
        "doc3": 0     # Not relevant
    },
    "query2": {
        "doc1": 0,    # Not relevant  
        "doc2": 1,    # Relevant - about Peter I founding St Petersburg
        "doc3": 0     # Not relevant
    },
    "query3": {
        "doc1": 0,    # Not relevant
        "doc2": 0,    # Not relevant
        "doc3": 1     # Relevant - about Lake Baikal being deepest
    },
    "query4": {
        "doc1": 0,    # Not relevant
        "doc2": 0,    # Partially relevant but not about lakes
        "doc3": 0     # Not relevant
    }
}

retriever.evaluate(qrels, results,k_values=[1,10,100])
retriever.evaluate_custom(qrels, results, k_values=[1,10,100], metric="mrr")

{'MRR@1': 0.25, 'MRR@10': 0.25, 'MRR@100': 0.25}

In [17]:
!curl -XGET 'http://127.0.0.1:9200/mmarco/_search?pretty=true'

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 3,
      "relation" : "eq"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "mmarco",
        "_id" : "doc1",
        "_score" : 1.0,
        "_source" : {
          "refresh" : "wait_for",
          "text" : "Озеро Байкал отличается уникальной флорой. В озере обитает более 2500 видов растений, из которых 1000 являются эндемиками."
        }
      },
      {
        "_index" : "mmarco",
        "_id" : "doc2",
        "_score" : 1.0,
        "_source" : {
          "refresh" : "wait_for",
          "text" : "В водах Байкала обитает множество уникальных животных. Байкальская нерпа и голомянка являются эндемиками озера."
        }
      },
      {
        "_index" : "mmarco",
        "_id" : "doc3",
        "_score" : 1.0,
        "_source" : {
          "refresh" : "wait_for",
          

In [16]:
!curl -XGET 'http://127.0.0.1:9200/_cat/indices?v'


health status index         uuid                   pri rep docs.count docs.deleted store.size pri.store.size dataset.size
yellow open   msmarco_index JArDdoPLSWehVkDHdXuqfQ   1   1          0            0       249b           249b         249b
yellow open   mmarco        UFaX_ZhLR8CCfZlG5FSkLA   1   1          3            0      6.3kb          6.3kb        6.3kb


In [18]:
!curl -XGET 'http://127.0.0.1:9200/mmarco/_mapping?pretty=true'


{
  "mmarco" : {
    "mappings" : {
      "properties" : {
        "refresh" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "text" : {
          "type" : "text"
        }
      }
    }
  }
}
